# OpenML Regression Benchmark 2025 – **Complete Notebook**

This notebook benchmarks a variety of regression models on the `New_OpenML_Suite_2025_regression` collection, logs experiments to **Weights & Biases**, runs hyper‑parameter sweeps, and tracks full data/model lineage.


In [ ]:
# ---- Setup ------------------------------------------------------------------
%pip install --upgrade -openml wandb scikit-learn optuna-integration[sklearn] pandas nbformat numpy joblib setuptools

import warnings, uuid, joblib
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import openml
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import cross_validate, KFold, train_test_split
from sklearn.metrics import (
    make_scorer,
    r2_score,
    mean_absolute_error,
    mean_squared_error,
)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.inspection import permutation_importance

from IPython.display import display
from joblib import Memory
import wandb

# Authenticate W&B (expects env var or local API key)
wandb.login()

wandb: Currently logged in as: spicecat (spicecat-club) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# ---- Configuration ----------------------------------------------------------
SUITE_NAME = "8f0ea660163b436bbd4abd49665c7b1d"  # OpenML-CTR23 - A curated tabular regression benchmarking suite
WANDB_PROJECT = "openml_regression_benchmark_2025"
CV_FOLDS = 5
RANDOM_STATE = 42

scoring = {
    "r2": make_scorer(r2_score),
    "mae": make_scorer(mean_absolute_error, greater_is_better=False),
    "mse": make_scorer(mean_squared_error, greater_is_better=False),
}

In [3]:
# ---- Helper • preprocessing --------------------------------------------------
def make_preprocessing_pipeline(X: pd.DataFrame):
    """Return ColumnTransformer that imputes numeric / categorical and outputs dense arrays."""
    numeric_features = X.select_dtypes(include=["number"]).columns.tolist()
    categorical_features = X.select_dtypes(exclude=["number"]).columns.tolist()

    numeric_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler()),
        ]
    )

    categorical_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            (
                "encoder",
                OneHotEncoder(handle_unknown="ignore", sparse_output=False, max_categories=20),
            ),
        ]
    )

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ],
        sparse_threshold=0.0,  # guarantee dense output
    )
    return preprocessor

In [4]:
# ---- Helper • reproducibility & lineage -------------------------------------
def log_dataset_artifact(run, dataset):
    art = wandb.Artifact(dataset.name.replace(" ", "_"), type="dataset")
    # add reference so artifact links back to OpenML
    if getattr(dataset, "url", None):
        art.add_reference(dataset.url)
    run.use_artifact(art)


def save_model_artifact(run, pipe_fitted, dataset_name, model_name):
    path = (
        f"/tmp/{dataset_name.replace(' ', '_')}_{model_name}_{uuid.uuid4().hex}.joblib"
    )
    joblib.dump(pipe_fitted, path)
    art = wandb.Artifact(f"{dataset_name}-{model_name}", type="model")
    art.add_file(path)
    run.log_artifact(art)

In [5]:
# ---- Helper • W&B plotting ---------------------------------------------------
def log_wandb_regressor_plots(
    pipe_fitted, X_train, X_test, y_train, y_test, model_name
):
    try:
        pre = pipe_fitted.named_steps.get("pre")
        if pre is not None:
            X_train_enc = pre.transform(X_train)
            X_test_enc = pre.transform(X_test)
            if hasattr(X_train_enc, "toarray"):
                X_train_enc = X_train_enc.toarray()
                X_test_enc = X_test_enc.toarray()
            reg = pipe_fitted.named_steps["model"]
        else:
            X_train_enc, X_test_enc = X_train, X_test
            reg = pipe_fitted

        wandb.sklearn.plot_regressor(
            reg, X_train_enc, X_test_enc, y_train, y_test, model_name=model_name
        )
        wandb.sklearn.plot_learning_curve(
            pipe_fitted, X_train_enc, y_train, cv=5, model_name=model_name
        )
    except Exception as exc:
        print(f"[W&B plots skipped] {exc}")

In [6]:
# ---- Helper • permutation importance ----------------------------------------
def log_permutation_importance(pipe_fitted, X_val, y_val, model_name):
    try:
        pre = pipe_fitted.named_steps.get("pre")
        X_val_enc = pre.transform(X_val) if pre else X_val
        if hasattr(X_val_enc, "toarray"):
            X_val_enc = X_val_enc.toarray()
        res = permutation_importance(
            pipe_fitted,
            X_val_enc,
            y_val,
            n_repeats=5,
            random_state=RANDOM_STATE,
            n_jobs=-1,
        )
        wandb.log({f"perm_imp_{model_name}": res.importances_mean})
    except Exception as exc:
        print(f"[perm importance skipped] {exc}")

In [ ]:
# ---- Helper • evaluation & W&B logging --------------------------------------
import copy


def evaluate_model(model_name: str, model, X, y):
    memory = Memory(location="/tmp/sk_cache", verbose=0)
    pre = make_preprocessing_pipeline(X)
    pipe = Pipeline(memory=memory, steps=[("pre", pre), ("model", model)])

    cv = KFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    cv_results = cross_validate(
        pipe, X, y, cv=cv, scoring=scoring, return_train_score=False, n_jobs=-1
    )

    metrics = {
        "r2": np.mean(cv_results["test_r2"]),
        "mae": -np.mean(cv_results["test_mae"]),
        "mse": -np.mean(cv_results["test_mse"]),
    }

    # Hold‑out split for plots & importance
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE
    )
    pipe_fit = Pipeline(
        memory=memory, steps=[("pre", pre), ("model", copy.deepcopy(model))]
    )
    pipe_fit.fit(X_train, y_train)

    # log_wandb_regressor_plots(pipe_fit, X_train, X_test, y_train, y_test, model_name)
    # log_permutation_importance(pipe_fit, X_test, y_test, model_name)

    return metrics, pipe_fit

In [8]:
# ---- Helper • sweep model builder -------------------------------------------
def make_model_from_sweep(cfg):
    if cfg.model == "RandomForest":
        return RandomForestRegressor(
            n_estimators=cfg.rf_n_estimators,
            max_depth=cfg.rf_max_depth,
            random_state=RANDOM_STATE,
        )
    if cfg.model == "MLP":
        return MLPRegressor(
            hidden_layer_sizes=tuple(cfg.mlp_layers),
            learning_rate_init=cfg.mlp_lr,
            max_iter=cfg.mlp_max_iter,
            random_state=RANDOM_STATE,
        )
    raise ValueError(cfg.model)

In [9]:
# ---- Benchmark loop – per‑dataset evaluation --------------------------------
results = []
suite = openml.study.get_suite(SUITE_NAME)
print(f"Evaluating {len(suite.tasks)} tasks from suite '{SUITE_NAME}'")

# Baseline models
baseline_models = {
    "LinearRegression": LinearRegression(),
    "RandomForest": RandomForestRegressor(n_estimators=200, random_state=RANDOM_STATE),
}

for task_id in suite.tasks[:2]:
    task = openml.tasks.get_task(task_id)
    dataset = task.get_dataset()
    X, y, *_ = dataset.get_data(
        target=dataset.default_target_attribute, dataset_format="dataframe"
    )

    print(f"\nDataset: {dataset.name} (task_id={task_id}) | shape={X.shape}")
    for name, model in baseline_models.items():
        run = wandb.init(
            project=WANDB_PROJECT,
            name=f"{dataset.name}-{name}",
            config={"dataset": dataset.name, "task_id": task_id, "model": name},
            reinit="finish_previous",
        )
        # log_dataset_artifact(run, dataset)

        try:
            metrics, pipe_fit = evaluate_model(name, model, X, y)
            wandb.log(metrics)
            # save_model_artifact(run, pipe_fit, dataset.name, name)
            results.append({"dataset": dataset.name, "model": name, **metrics})
        except Exception as exc:
            print(f"⚠️ {name} on {dataset.name} failed: {exc}")
            wandb.log({"error": str(exc)})
        finally:
            run.finish()

Evaluating 35 tasks from suite '8f0ea660163b436bbd4abd49665c7b1d'

Dataset: abalone (task_id=361234) | shape=(4177, 8)


mae,▁
mape,▁
mse,▁
r2,▁
mae,1.58555
mape,0.1598
mse,4.90657
r2,0.52571


mae,▁
mape,▁
mse,▁
r2,▁
mae,1.5517
mape,0.15493
mse,4.79935
r2,0.53579



Dataset: airfoil_self_noise (task_id=361235) | shape=(1503, 5)


mae,▁
mape,▁
mse,▁
r2,▁
mae,3.7542
mape,0.03016
mse,23.36428
r2,0.50514


mae,▁
mape,▁
mse,▁
r2,▁
mae,1.29732
mape,0.01047
mse,3.30809
r2,0.93015


In [10]:
# ---- Results summary ---------------------------------------------------------
df_results = pd.DataFrame(results)
display(df_results.head())

,dataset,model,r2,mae,mse,mape
0,abalone,LinearRegression,0.525713,1.585550,4.906571,0.159800
1,abalone,RandomForest,0.535787,1.551699,4.799352,0.154928
2,airfoil_self_noise,LinearRegression,0.505137,3.754198,23.364282,0.030157
3,airfoil_self_noise,RandomForest,0.930148,1.297316,3.308089,0.010470


## W&B Sweeps

Optimize hyper‑parameters across **datasets** and **models** with a Bayesian sweep.


In [11]:
# ---- Define and run a W&B sweep ---------------------------------------------
sweep_config = {
    "method": "bayes",
    "metric": {"name": "rmse", "goal": "minimize"},
    "parameters": {
        "dataset_idx": {"values": list(range(len(suite.tasks)))},
        "model": {"values": ["RandomForest", "MLP"]},
        "rf_n_estimators": {"values": [100, 300, 500]},
        "rf_max_depth": {"values": [None, 10, 20]},
        "gb_n_estimators": {"values": [100, 300]},
        "gb_lr": {"values": [0.05, 0.1, 0.2]},
        "mlp_layers": {"values": [(64,), (128, 64)]},
        "mlp_lr": {"values": [0.001, 0.01]},
        "mlp_max_iter": {"values": [200, 400]},
    },
}

sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT)


def sweep_train():
    with wandb.init() as run:
        cfg = wandb.config
        task = openml.tasks.get_task(suite.tasks[cfg.dataset_idx])
        ds = task.get_dataset()
        X_sw, y_sw, *_ = ds.get_data(
            target=ds.default_target_attribute, dataset_format="dataframe"
        )
        model = make_model_from_sweep(cfg)
        metrics, _ = evaluate_model(cfg.model, model, X_sw, y_sw)
        wandb.log(metrics)


# ➡️ Uncomment to launch 20 trials locally
wandb.agent(sweep_id, function=sweep_train, count=20)

Create sweep with ID: d1ru4g8r
Sweep URL: https://wandb.ai/spicecat-club/openml_regression_benchmark_2025/sweeps/d1ru4g8r


wandb: Agent Starting Run: f0bxpfuv with config:
wandb: 	dataset_idx: 27
wandb: 	gb_lr: 0.05
wandb: 	gb_n_estimators: 100
wandb: 	mlp_layers: [64]
wandb: 	mlp_lr: 0.001
wandb: 	mlp_max_iter: 200
wandb: 	model: MLP
wandb: 	rf_max_depth: None
wandb: 	rf_n_estimators: 300


/home/ahteh/Developer/research/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ahteh/Developer/research/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ahteh/Developer/research/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ahteh/Developer/research/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged ye

mae,▁
mape,▁
mse,▁
r2,▁
mae,6510.55463
mape,2.38699
mse,176348532.11954
r2,0.53471


wandb: Agent Starting Run: o3yf9pae with config:
wandb: 	dataset_idx: 21
wandb: 	gb_lr: 0.1
wandb: 	gb_n_estimators: 100
wandb: 	mlp_layers: [64]
wandb: 	mlp_lr: 0.01
wandb: 	mlp_max_iter: 400
wandb: 	model: MLP
wandb: 	rf_max_depth: 20
wandb: 	rf_n_estimators: 100


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: i5z0hzcn with config:
wandb: 	dataset_idx: 15
wandb: 	gb_lr: 0.2
wandb: 	gb_n_estimators: 300
wandb: 	mlp_layers: [64]
wandb: 	mlp_lr: 0.01
wandb: 	mlp_max_iter: 400
wandb: 	model: MLP
wandb: 	rf_max_depth: None
wandb: 	rf_n_estimators: 300


/home/ahteh/Developer/research/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ahteh/Developer/research/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ahteh/Developer/research/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ahteh/Developer/research/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged ye

mae,▁
mape,▁
mse,▁
r2,▁
mae,45579.99009
mape,0.26234
mse,4117247573.88396
r2,0.69078


wandb: Agent Starting Run: 294e383h with config:
wandb: 	dataset_idx: 6
wandb: 	gb_lr: 0.05
wandb: 	gb_n_estimators: 300
wandb: 	mlp_layers: [128, 64]
wandb: 	mlp_lr: 0.001
wandb: 	mlp_max_iter: 400
wandb: 	model: RandomForest
wandb: 	rf_max_depth: 20
wandb: 	rf_n_estimators: 500


mae,▁
mape,▁
mse,▁
r2,▁
mae,12.00533
mape,1.06944
mse,245.53834
r2,0.2743


## Appendix

Additional experiments and notes can go here.
